In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *

# Read data

In [2]:
y = read_target_train()

# Preprocess data

In [3]:
X, X_test = read_preprocessed_data()

# Train catboost

In [4]:
from catboost import CatBoostRegressor

In [5]:
model_tst = CatBoostRegressor(iterations = 400, verbose = 50, depth = 3, random_state = config['random_state'], cat_features=get_cat_features_list(X), loss_function='MAE')
model_C = CatBoostRegressor(iterations = 400, verbose = 50, depth = 3, random_state = config['random_state'], cat_features=get_cat_features_list(X), loss_function='MAE')

In [6]:
model_tst.fit(X, y['TST'])

0:	learn: 23.6791847	total: 61.8ms	remaining: 24.7s
50:	learn: 18.6059626	total: 722ms	remaining: 4.94s
100:	learn: 17.2868372	total: 1.38s	remaining: 4.08s
150:	learn: 16.4806965	total: 2.03s	remaining: 3.35s
200:	learn: 15.8925848	total: 2.68s	remaining: 2.66s
250:	learn: 15.3768116	total: 3.34s	remaining: 1.98s
300:	learn: 14.8784112	total: 3.99s	remaining: 1.31s
350:	learn: 14.3951568	total: 4.64s	remaining: 648ms
399:	learn: 14.0092125	total: 5.29s	remaining: 0us


In [7]:
model_C.fit(X, y['C'])

0:	learn: 0.0386910	total: 14.1ms	remaining: 5.63s
50:	learn: 0.0310657	total: 685ms	remaining: 4.68s
100:	learn: 0.0272749	total: 1.34s	remaining: 3.96s
150:	learn: 0.0232801	total: 1.99s	remaining: 3.27s
200:	learn: 0.0218192	total: 2.63s	remaining: 2.61s
250:	learn: 0.0209442	total: 3.29s	remaining: 1.95s
300:	learn: 0.0203070	total: 3.94s	remaining: 1.29s
350:	learn: 0.0199564	total: 4.6s	remaining: 642ms
399:	learn: 0.0196693	total: 5.24s	remaining: 0us


In [8]:
save_catboost_models(model_tst, model_C)

### Model Validation

In [9]:
from catboost import Pool, cv

cv_data = X

labels = y['TST']

cat_features = get_cat_features_list(X)

cv_dataset = Pool(data=cv_data,
                  label=labels,
                  cat_features=cat_features)

params = {"iterations": 1200,
          "depth": 3,
          "loss_function": "MAE",
          "verbose": 50}

scores = cv(cv_dataset,
            params,
            fold_count=5,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1599.5837018	test: 1599.5819651	best: 1599.5819651 (0)
50:	learn: 347.2068191	test: 347.4235684	best: 347.4235684 (50)
100:	learn: 74.6744672	test: 74.9504856	best: 74.9504856 (100)
150:	learn: 24.6377079	test: 25.1747337	best: 25.1747337 (150)


KeyboardInterrupt: 

In [ ]:
from catboost import Pool, cv

cv_data = X

labels = y['C']

cat_features = get_cat_features_list(X)

cv_dataset = Pool(data=cv_data,
                  label=labels,
                  cat_features=cat_features)

params = {"iterations": 1200,
          "depth": 3,
#           "l2_leaf_reg" : 4,
          "loss_function": "MAE",
          "verbose": 50}

scores = cv(cv_dataset,
            params,
            fold_count=5, 
            plot="True")